# CNN   small datasets 학습


- Data의 수가 많지 않을 때 CNN을 통한 모형 학습이 어려울 수 있음
  - 딥러닝은 많은 수의 데이터를 통해 feature engineering 과정 없이 feature를 찾을 수 있는데 있음 
  - class당 몇개의 이미지가 있는지도 확인 해야한다.
- Data가 많지 않아 CNN 학습에 어려움이 있을 때 사용 가능한 방법
    - Data augmentation 활용
        - 이미지의 색깔, 각도 등을 약간씩 변형하여 data의 수를 늘림 
    - Pre-trained network의 활용
        - 매우 큰 데이터셋으로 미리 Training한 모델의 파라미터(가중치)를 가져와서 풀려는 문제에 맞게 모델을 재보정해서 사용하는 것.
        - 미리 다양한 데이터를 가지고 학습된 모델을 사용하므로 적은 데이터에도 좋은 성능을 낼 수있다.

## Data for cats vs. dogs
- 2013년 Kaggle의 computer vision competition data 활용 https://www.kaggle.com/c/dogs-vs-cats/data
- 개와 고양이를 구분하기 위한 문제로 각 12,500개의 이미지를 포함
- Medium-resolution color JPEGs
- 25000장의 사진 중 4000장의 cats/dogs 사진(2000 cats, 2000 dogs) 만을 사용하여 학습하여 좋은 모형을 만들어 낼 수 있을까?
    - 학습: 2000, 검증: 1000, 테스트: 1000
    
![cats_vs_dogs_samples](https://s3.amazonaws.com/book.keras.io/img/ch5/cats_vs_dogs_samples.jpg)

- gdown 패키지 : 구글 드라이브의 공유파일 다운로드 패키지    
- `pip install gdown==3.3.1`
- 코랩에는 설치 되어 있음.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 이미지 다운로드
#https://drive.google.com/uc?id= 공유파일 ID

import gdown
url = 'https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU'
fname = 'cats_and_dogs_small.zip'


gdown.download(url, fname, quiet = False) # 다운받을 곳 url, 저장경로 //quiet = False  다운받은 로그 남기기.. True면 안남긴다.

Downloading...
From: https://drive.google.com/uc?id=1nBE3N2cXQGwD8JaD0JZ2LmFD-n3D5hVU
To: /content/cats_and_dogs_small.zip
90.8MB [00:00, 98.5MB/s]


'cats_and_dogs_small.zip'

In [ ]:
# 리눅스 명령어로 디렉토리 생성
!mkdir data

In [ ]:
# 압축 풀기 -q : 로그남기지 말아라. -d 압축을 어디에 풀것인지 디렉토리 지정.
!unzip -q cats_and_dogs_small.zip -d data/cats_and_dogs_small

## Build a network

- Input: $150 \times 150$ 픽셀의 RGB layer 
- Output: cat or dog (binary classification) 
- ImageDataGenerator를 이용해 파일시스템에 저장된 이미지데이터셋을 학습시킨다.

In [40]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

np.random.seed(1)
tf.random.set_seed(1)

In [41]:
# 하이퍼 파라미터
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
N_EPOCHS = 50
N_BATCHS = 20
IMAGE_SIZE = 150


In [42]:
def create_model():
    model = keras.Sequential()
    model.add(layers.Input((IMAGE_SIZE,IMAGE_SIZE,3)))
    

    model.add(layers.Conv2D(filters = 64, kernel_size= 3, padding = 'same', activation= 'relu'))
    model.add(layers.MaxPool2D(padding = 'same'))

    model.add(layers.Conv2D(filters = 128, kernel_size= 3, padding= 'same', activation= 'relu'))
    model.add(layers.MaxPool2D(padding= 'same'))

    model.add(layers.Conv2D(filters = 256, kernel_size= 3, padding= 'same', activation= 'relu'))
    model.add(layers.MaxPool2D(padding= 'same'))


    #classification

    model.add(layers.Flatten())
    model.add(layers.Dropout(DROPOUT_RATE))
    model.add(layers.Dense(units = 512, activation= 'relu'))
    # 출력

    model.add(layers.Dense(units = 1, activation= 'sigmoid')) # dog / cat : 이진분류.. 

    return model 

In [ ]:
model = create_model()
model.compile(optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 64)      1792      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 38, 38, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 38, 38, 256)       295168    
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 19, 19, 256)       0         
_________________________________________________________________
flatten (Flatten)            (None, 92416)             0

In [ ]:
# imageDataGenerate 생성 -> Augmentation , 입력 pipeline
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt

test_dir = '/content/data/cats_and_dogs_small/test'
train_dir = '/content/data/cats_and_dogs_small/train'
validation_dir ='/content/data/cats_and_dogs_small/validation'

In [ ]:
# 1. ImageDataGenerator - no Augmentation
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
# Generator.flow_from_directiory() 이용해서 iterator 생성
train_iterator = train_datagen.flow_from_directory(directory = train_dir, #이미지들의 디렉토리.
                                                   target_size =(IMAGE_SIZE,IMAGE_SIZE), #채널은 안넣음. Resize크기(height,width)
                                                   class_mode ='binary', # dog/cat ->binary
                                                   batch_size = N_BATCHS) # 배치사이즈는 메모리 양에 따리 조절 

validation_iterator= test_datagen.flow_from_directory(directory= validation_dir,
                                                      target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                      class_mode ='binary',
                                                      batch_size = N_BATCHS)

test_iterator = test_datagen.flow_from_directory(directory= test_dir,
                                                      target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                      class_mode ='binary',
                                                      batch_size = N_BATCHS)


Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
train_iterator.class_indices

{'cats': 0, 'dogs': 1}

In [ ]:
len(train_iterator), len(validation_iterator), len(test_iterator) # 1에폭당 step수

(100, 50, 50)


##  Model Training(학습)

In [ ]:
history = model.fit(train_iterator,
                    epochs = N_EPOCHS,
                    steps_per_epoch = len(train_iterator),
                    validation_data = validation_iterator,
                    validation_steps = len(validation_iterator))

# 이미지의 양이 너무 적음.. 오버 피팅... 드랍아웃 까지 했는데..
# augmentation  사용안함 

Epoch 1/50
100/100 [==============================] - 43s 95ms/step - loss: 1.1893 - accuracy: 0.5097 - val_loss: 0.6896 - val_accuracy: 0.4990
Epoch 2/50
100/100 [==============================] - 9s 91ms/step - loss: 0.6795 - accuracy: 0.5406 - val_loss: 0.6888 - val_accuracy: 0.5320
Epoch 3/50
100/100 [==============================] - 9s 92ms/step - loss: 0.6903 - accuracy: 0.5342 - val_loss: 0.6859 - val_accuracy: 0.5730
Epoch 4/50
100/100 [==============================] - 9s 90ms/step - loss: 0.6611 - accuracy: 0.6084 - val_loss: 0.6489 - val_accuracy: 0.5910
Epoch 5/50
100/100 [==============================] - 9s 90ms/step - loss: 0.6288 - accuracy: 0.6331 - val_loss: 0.6209 - val_accuracy: 0.6660
Epoch 6/50
100/100 [==============================] - 9s 91ms/step - loss: 0.6193 - accuracy: 0.6731 - val_loss: 0.6164 - val_accuracy: 0.6720
Epoch 7/50
100/100 [==============================] - 9s 91ms/step - loss: 0.5612 - accuracy: 0.6916 - val_loss: 0.6320 - val_accuracy: 0.659

In [ ]:
model.evaluate(test_iterator)

50/50 [==============================] - 3s 54ms/step - loss: 2.6660 - accuracy: 0.6900


[2.666003942489624, 0.6899999976158142]

- Overfitting발생
    - 적은 이미지 수.. 

# Using data augmentation
- image augmentation을 하기위해 pipeline/ 한번에 처리 하기위해 사용하는게 ImageDataGenerator 이다.
- 학습 이미지의 수가 적어서 overfitting이 발생할 가능성을 줄이기 위해 기존 훈련 데이터로부터 그럴듯하게 이미지 변환을 통해서 이미지(데이터)를 늘리는 작업을 Image augmentation
- train_set에만 적용, validation, test set에는 적용하지 않는다. (rescaling만 한다.)

In [ ]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range =(0.7,1.3),
                                   fill_mode = 'constant') # 트레인은 데이터 양이 부족하니까 증식 했던것이다...
# 이미지가 어떻게 변하는지 먼저확인하기 위해서 rescale을 일단 주석처리 햇었음..
# validation, test 용

test_datagen = ImageDataGenerator(rescale= 1./255)  # 검증하고 테스트만 할건데 늘릴(Augmentation) 이유가 없다...

In [ ]:
train_iterator = train_datagen.flow_from_directory(train_dir,
                                                   target_size =(IMAGE_SIZE,IMAGE_SIZE), # 차원 안넣음
                                                   class_mode ='binary',
                                                   batch_size = N_BATCHS)


validation_iterator = test_datagen.flow_from_directory(validation_dir,
                                                        target_size =(IMAGE_SIZE,IMAGE_SIZE),
                                                        class_mode ='binary',
                                                        batch_size = N_BATCHS)

test_iterator = test_datagen.flow_from_directory(test_dir,
                                                   target_size =(IMAGE_SIZE,IMAGE_SIZE),
                                                   class_mode ='binary',
                                                   batch_size = N_BATCHS)

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [ ]:
# 이미지 확인

batch_image = train_iterator.next()
batch_image[0].shape # batch_image[0]- 이미지,, ,, batch_image[1]- label 

(20, 150, 150, 3)

In [ ]:
batch_image[1].shape  # 배치사이즈가 20이라서 

(20,)

In [ ]:
plt.figure(figsize=(30,15))
for i in range(20):
    plt.subplot(4,5,i+1)
    img = batch_image[0][i].astype('uint8') # 아그먼테이션을 하면 float 타입으로 나오기 때문에.. 
    # 왜 batch-image의 [0]번을 하는 지모르겠음
    plt.imshow(img)  # int값으로 들어가야됨..
    plt.axis('off')

plt.tight_layout()
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
model2 = create_model()
model2.compile(optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE),loss= 'binary_crossentropy',metrics =['accuracy'])

In [ ]:
model2.fit(train_iterator,
           epochs = N_EPOCHS,
           steps_per_epoch = len(train_iterator), # trian_iterator는 배치 사이즈 만큼 이미지를 모델에 제공하니까.
           validation_data = validation_iterator,
           validation_steps = len(validation_iterator))

# 이미지 증식 작업때문에 시간이 좀 오래 걸린다... 


Epoch 1/50
100/100 [==============================] - 21s 201ms/step - loss: 1.1875 - accuracy: 0.5266 - val_loss: 0.6903 - val_accuracy: 0.5540
Epoch 2/50
100/100 [==============================] - 18s 184ms/step - loss: 0.6940 - accuracy: 0.5048 - val_loss: 0.6884 - val_accuracy: 0.5720
Epoch 3/50
100/100 [==============================] - 19s 185ms/step - loss: 0.6895 - accuracy: 0.5293 - val_loss: 0.6925 - val_accuracy: 0.5500
Epoch 4/50
100/100 [==============================] - 19s 186ms/step - loss: 0.6946 - accuracy: 0.4950 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 5/50
100/100 [==============================] - 18s 184ms/step - loss: 0.6933 - accuracy: 0.4901 - val_loss: 0.6928 - val_accuracy: 0.5000
Epoch 6/50
100/100 [==============================] - 19s 186ms/step - loss: 0.6933 - accuracy: 0.5104 - val_loss: 0.6920 - val_accuracy: 0.5020
Epoch 7/50
100/100 [==============================] - 19s 185ms/step - loss: 0.6944 - accuracy: 0.4886 - val_loss: 0.6927 - val_ac

- train, validation 이 거의 같은 비율로 정확성이 올라가고 있다..
- 에폭을 좀 더 돌렸을때 더 나아 질 수도 있다..... 
- augmentation 을 사용하면 과적합을 어느 정도 해결을 가능 할 수 있다( 데이터 수가 늘어나면서..)
- 이미지 변형을 계속 하면서 학습하기때문에 시간은 좀 더 걸린다.

###  DataFrame 이용
- flow_from_dataframe() 사용
    - 파일경로와 label을 DataFrame으로 저장하고 그것을 이용해 데이터셋을 읽어온다.

In [1]:
import gdown

url = 'https://drive.google.com/uc?id=17ejPJw42TgTv0jCPMMlVTHwF57XYE2kb'
fname = 'cats_and_dogs_union.zip'
gdown.download(url, fname, quiet = True) # 다운 받아짐..

'cats_and_dogs_union.zip'

In [2]:
!mkdir data # data 폴더를 만들었고 이곳에 압축을 풀것임..

In [3]:
!unzip -q ./cats_and_dogs_union.zip -d ./data/cats_and_dogs  #압축이 플렸음..
# 디렉토리는 폴더별로 train,test,label 로 나뉘어 져있었지만
# dataframe으로 하때는 cats와 dogs로만 나뉘어져 있다..

In [4]:
import pandas as pd

# DAtaFrame 생성
- path, label 컬럼을 만들어야함.

In [6]:
# 파일 경로 다루기 - glob
from glob import glob
# ** : 모든 하위경로를 다 찾으셈~! / *.jpg(확장자가 jpg인 모든파일)
path_list = glob('/content/data/cats_and_dogs/**/*.jpg') #모든 하위 경로에서 모든 .jpg 파일의 경로를 찾아라
# 지정한 파일들의 absolute path(절대경로)를 문자열로 반환(리스트에 담아서반환.)
len(path_list)

4000

In [7]:
path_list[:10]

['/content/data/cats_and_dogs/cats/cat.1303.jpg',
 '/content/data/cats_and_dogs/cats/cat.1974.jpg',
 '/content/data/cats_and_dogs/cats/cat.1335.jpg',
 '/content/data/cats_and_dogs/cats/cat.766.jpg',
 '/content/data/cats_and_dogs/cats/cat.1203.jpg',
 '/content/data/cats_and_dogs/cats/cat.1351.jpg',
 '/content/data/cats_and_dogs/cats/cat.151.jpg',
 '/content/data/cats_and_dogs/cats/cat.1079.jpg',
 '/content/data/cats_and_dogs/cats/cat.768.jpg',
 '/content/data/cats_and_dogs/cats/cat.1916.jpg']

In [8]:
path_list[-10:]

['/content/data/cats_and_dogs/dogs/dog.1999.jpg',
 '/content/data/cats_and_dogs/dogs/dog.1812.jpg',
 '/content/data/cats_and_dogs/dogs/dog.486.jpg',
 '/content/data/cats_and_dogs/dogs/dog.1908.jpg',
 '/content/data/cats_and_dogs/dogs/dog.603.jpg',
 '/content/data/cats_and_dogs/dogs/dog.1585.jpg',
 '/content/data/cats_and_dogs/dogs/dog.1375.jpg',
 '/content/data/cats_and_dogs/dogs/dog.575.jpg',
 '/content/data/cats_and_dogs/dogs/dog.577.jpg',
 '/content/data/cats_and_dogs/dogs/dog.1948.jpg']

In [12]:
import os 

f= '/content/data/cats_and_dogs/dogs/dog.1999.jpg'
print(os.path.basename(f)) # basename(경로):경로에서 파일명만 조회

print(os.path.dirname(f))# dirname(경로):경로에서 디렉토리 부분만 조회

# 여기서는 디렉토리 네임(dogs,cats)이 라벨이 될거임../ 

print(os.path.dirname(f).split(r'/')[4])  

dog.1999.jpg
/content/data/cats_and_dogs/dogs
dogs


In [13]:
label_list = []

for path in path_list:
    l = os.path.dirname(path).split(r'/')[4]
    label_list.append(l)

In [15]:
label_list = [os.path.dirname(path).split(r'/')[4] for path in path_list]
label_list[:5], label_list[-5:], len(label_list)

(['cats', 'cats', 'cats', 'cats', 'cats'],
 ['dogs', 'dogs', 'dogs', 'dogs', 'dogs'],
 4000)

In [17]:
d ={
    "path":path_list,
    "label":label_list
}
data_df = pd.DataFrame(d)

data_df.shape

(4000, 2)

In [18]:
data_df.head()

,path,label
0,/content/data/cats_and_dogs/cats/cat.1303.jpg,cats
1,/content/data/cats_and_dogs/cats/cat.1974.jpg,cats
2,/content/data/cats_and_dogs/cats/cat.1335.jpg,cats
3,/content/data/cats_and_dogs/cats/cat.766.jpg,cats
4,/content/data/cats_and_dogs/cats/cat.1203.jpg,cats


In [19]:
data_df.tail()

,path,label
3995,/content/data/cats_and_dogs/dogs/dog.1585.jpg,dogs
3996,/content/data/cats_and_dogs/dogs/dog.1375.jpg,dogs
3997,/content/data/cats_and_dogs/dogs/dog.575.jpg,dogs
3998,/content/data/cats_and_dogs/dogs/dog.577.jpg,dogs
3999,/content/data/cats_and_dogs/dogs/dog.1948.jpg,dogs


In [20]:
data_df['label'].value_counts()

dogs    2000
cats    2000
Name: label, dtype: int64

In [22]:
data_df.to_csv('./data/cats_and_dogs_filelist.csv', encoding= 'utf-8', index =None)
#만든 DataFrame을 저장해서 사용할거임~!

In [25]:
# cats, dogs DataFrame으로 분리/
#앞쪽 2000,뒤쪽 2000 cat,dog로 나뉘어져 있어서 train,test df을 만들기 위해서 나눔..

cats_df=data_df[data_df['label'] =='cats']
dogs_df=data_df[data_df['label'] =='dogs']

In [27]:
cats_df.shape, dogs_df.shape

((2000, 2), (2000, 2))

In [32]:
# train/test dataframe을 생성 8:2 로 
split_idx = int(dogs_df.shape[0]*0.8)

In [38]:
train_df = pd.concat([dogs_df[:split_idx],cats_df[:split_idx]], axis = 0) # dog, cats의 0~1599개를 묶어서 train_df 생성
#dogs_df[split_idx:] # 나머지 20 프로
train_df['label'].value_counts()

dogs    1600
cats    1600
Name: label, dtype: int64

In [37]:
test_df = pd.concat([dogs_df[split_idx:], cats_df[split_idx:]], axis =0)

print(test_df.shape)
test_df['label'].value_counts()

(800, 2)


dogs    400
cats    400
Name: label, dtype: int64

In [39]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range = 40,
                                   width_shift_range = 0.1,
                                   height_shift_range = 0.1,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range =(0.7,1.3),
                                   fill_mode = 'constant') # 트레인은 데이터 양이 부족하니까 증식 했던것이다...
# 이미지가 어떻게 변하는지 먼저확인하기 위해서 rescale을 일단 주석처리 햇었음..
# validation, test 용

test_datagen = ImageDataGenerator(rescale= 1./255)

In [49]:
train_iterator = train_datagen.flow_from_dataframe(dataframe= train_df, # path와 라벨을 가진 DataFrame 객체 지정
                                                   x_col ='path', # 이미지경로를 가진 컬럼명
                                                   y_col ='label', # label 컬럼명
                                                   target_size = (IMAGE_SIZE,IMAGE_SIZE),
                                                   class_mode ='binary',
                                                   batch_size = N_BATCHS)



Found 3200 validated image filenames belonging to 2 classes.


In [50]:
test_iterator = test_datagen.flow_from_dataframe(test_df,
                                                 x_col = 'path',
                                                 y_col ='label',
                                                 target_size =(IMAGE_SIZE,IMAGE_SIZE),
                                                 class_mode = 'binary',
                                                 batch_size = N_BATCHS)

Found 800 validated image filenames belonging to 2 classes.


In [45]:
train_df.head()

,path,label
2000,/content/data/cats_and_dogs/dogs/dog.1348.jpg,dogs
2001,/content/data/cats_and_dogs/dogs/dog.1506.jpg,dogs
2002,/content/data/cats_and_dogs/dogs/dog.1699.jpg,dogs
2003,/content/data/cats_and_dogs/dogs/dog.182.jpg,dogs
2004,/content/data/cats_and_dogs/dogs/dog.344.jpg,dogs


In [51]:
train_iterator.class_indices

{'cats': 0, 'dogs': 1}

In [52]:
model = create_model()
model.compile(optimizer = keras.optimizers.Adam(learning_rate= LEARNING_RATE),
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 150, 150, 64)      1792      
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 75, 75, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 75, 75, 128)       73856     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 38, 38, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 38, 38, 256)       295168    
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 19, 19, 256)       0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 92416)            

In [53]:
model.fit(train_iterator,
          epochs = N_EPOCHS,
          steps_per_epoch = len(train_iterator),
          validation_data = test_iterator,
          validation_steps = len(test_iterator))

Epoch 1/50
160/160 [==============================] - 60s 170ms/step - loss: 1.0415 - accuracy: 0.4986 - val_loss: 0.6926 - val_accuracy: 0.5312
Epoch 2/50
160/160 [==============================] - 27s 170ms/step - loss: 0.6932 - accuracy: 0.5024 - val_loss: 0.6931 - val_accuracy: 0.5000
Epoch 3/50
160/160 [==============================] - 27s 171ms/step - loss: 0.6933 - accuracy: 0.5269 - val_loss: 0.6881 - val_accuracy: 0.6125
Epoch 4/50
160/160 [==============================] - 27s 170ms/step - loss: 0.6912 - accuracy: 0.5331 - val_loss: 0.6657 - val_accuracy: 0.6050
Epoch 5/50
160/160 [==============================] - 28s 172ms/step - loss: 0.6891 - accuracy: 0.5430 - val_loss: 0.6698 - val_accuracy: 0.5800
Epoch 6/50
160/160 [==============================] - 27s 171ms/step - loss: 0.6837 - accuracy: 0.5648 - val_loss: 0.6703 - val_accuracy: 0.5575
Epoch 7/50
160/160 [==============================] - 27s 171ms/step - loss: 0.6770 - accuracy: 0.5890 - val_loss: 0.6747 - val_ac

## 추론

In [54]:
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [66]:
def pred_cat_dog(path):
    class_name =['cat', 'dog']
    img = load_img(path, target_size=(IMAGE_SIZE,IMAGE_SIZE))

    # image --> ndarray
    sample = img_to_array(img)[np.newaxis,...] # 이미지 개수가 들어갈 축을 늘려줘야해서 
    # scaling
    sample = sample/255.

    pred = model.predict(sample) # 확률
    print(pred)
    pred = pred[0,0]
    print(pred)
    pred_class = np.where(pred<0.5, 0 ,1)
    print(pred_class)
    pred_class_name = class_name[pred_class]
    return pred, pred_class, pred_class_name

In [67]:
pred_cat_dog('/content/cat.jpg')

(0.43034568, array(0), 'cat')

In [68]:
pred_cat_dog('/content/dog.jpg')

(0.6611469, array(1), 'dog')